In [1]:
import sys
sys.path.append('/home/linxuangao')  # 确保父目录在 Python 路径中

In [2]:
import pandas as pd
import numpy as np
import scanpy as sc
import anndata as ad
import torch
import umap
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import scipy

from GPNSF.model import GPNSFModel
from GPNSF.utils import *


import warnings
warnings.filterwarnings("ignore")

/home/linxuangao/.conda/envs/glx/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
print(f'Using device: {device}')

Using device: cuda


In [4]:
data_dir = '/home/linxuangao/data_gpnsf/P22 mouse brain'

In [5]:
adata1 = sc.read_h5ad(f'{data_dir}/P22_mouse_brain_adata_RNA.h5ad')
adata2 = sc.read_h5ad(f'{data_dir}/P22_mouse_brain_adata_atac.h5ad')

In [6]:
print(adata1.X.shape, adata2.X.shape)  # (N, D1), (N, D2)
print(adata1.n_vars, adata2.n_vars)  # D1, D2

(9215, 22914) (9215, 121068)
22914 121068


In [7]:
def preprocess_multiome_data(adata_rna, adata_atac, 
                             rna_n_features=3000,
                             atac_n_features=10000):
    """多组学数据预处理流程"""
    
    print("=" * 50)
    print("RNA数据预处理")
    print("=" * 50)
    
    # RNA数据预处理
    # 1. 基本质量控制
    sc.pp.filter_genes(adata_rna, min_counts=50)
    print(f"RNA: 过滤后基因数: {adata_rna.n_vars}")
    
    # 2. 选择高变基因
    sc.pp.highly_variable_genes(
        adata_rna, 
        n_top_genes=rna_n_features,
        flavor='seurat_v3'
    )
    adata_rna = adata_rna[:, adata_rna.var['highly_variable']].copy()
    print(f"RNA: 选择 {adata_rna.n_vars} 个高变基因")
    
    # 3. 标准化
    sc.pp.normalize_total(adata_rna, target_sum=1e4)
    sc.pp.log1p(adata_rna)
    
    print("\n" + "=" * 50)
    print("ATAC数据预处理")
    print("=" * 50)
    
    # ATAC数据预处理
    # 1. 基本质量控制
    sc.pp.filter_genes(adata_atac, min_cells=50)
    print(f"ATAC: 过滤后peak数: {adata_atac.n_vars}")
    
    # 2. 选择高变peak
    sc.pp.highly_variable_genes(
        adata_atac, 
        n_top_genes=atac_n_features,
        flavor='seurat_v3'
    )
    adata_atac = adata_atac[:, adata_atac.var['highly_variable']].copy()
    print(f"ATAC: 选择 {adata_atac.n_vars} 个高变peak")
    
    # # 3. 标准化（ATAC通常使用TF-IDF）
    # from sklearn.feature_extraction.text import TfidfTransformer
    
    # if scipy.sparse.issparse(adata_atac.X):
    #     X = adata_atac.X.toarray()
    # else:
    #     X = adata_atac.X
    
    # tfidf = TfidfTransformer()
    # adata_atac.X = tfidf.fit_transform(X)
    
    print("\n预处理完成!")
    print(f"RNA矩阵大小: {adata_rna.shape}")
    print(f"ATAC矩阵大小: {adata_atac.shape}")
    
    return adata_rna, adata_atac

# 使用示例
adata1_filtered, adata2_filtered = preprocess_multiome_data(
    adata1.copy(), adata2.copy(),
    rna_n_features=3000,
    atac_n_features=10000
)

RNA数据预处理
RNA: 过滤后基因数: 13214
RNA: 选择 3000 个高变基因

ATAC数据预处理
ATAC: 过滤后peak数: 120400
ATAC: 选择 10000 个高变peak

预处理完成!
RNA矩阵大小: (9215, 3000)
ATAC矩阵大小: (9215, 10000)


In [8]:
X_1 = adata1_filtered.X
p = X_1.shape[1]
X_2 = adata2_filtered.X
q = X_2.shape[1]
S = adata1.obsm['spatial']

del adata1, adata2  # 释放内存

In [9]:
K = 16
M = 128

In [15]:
# 重新加载 GPNSF 并将 numpy/sparse 转为 torch 张量
from importlib import reload
import GPNSF.model as gp_model
reload(gp_model)
from GPNSF.model import GPNSFModel

import numpy as np
import scipy.sparse as sp

def to_torch_tensor(x, device, dtype=torch.float32):
    if hasattr(x, "toarray"):
        x = x.toarray()  # scipy.sparse -> ndarray
    if isinstance(x, np.ndarray):
        return torch.as_tensor(x, dtype=dtype, device=device)
    if isinstance(x, torch.Tensor):
        return x.to(device=device, dtype=dtype)
    raise TypeError(f"Unsupported type: {type(x)}")

S_t = to_torch_tensor(S, device, dtype=torch.float32)
X1_t = to_torch_tensor(X_1, device, dtype=torch.float32)
X2_t = to_torch_tensor(X_2, device, dtype=torch.float32)

print('Converted ->', 'S:', S_t.shape, 'X1:', X1_t.shape, 'X2:', X2_t.shape, 'device=', device)

Converted -> S: torch.Size([9215, 2]) X1: torch.Size([9215, 3000]) X2: torch.Size([9215, 10000]) device= cuda


In [ ]:
# 构建模型与训练循环（周期性打印 loss/ELBO/KL）
model = GPNSFModel(S=S_t, p=p, q=q, K=K, M=M, eta=1.0, num_mc_samples=3, likelihood_x2='ber', 
                   omega_type='chol', kernel_type='matern32').to(device)

opt = torch.optim.Adam(model.parameters(), lr=1e-4)

@torch.no_grad()
def compute_loglik_terms(model, X1, X2):
    # 与模型内部保持一致的 Monte Carlo 估计，并适配 X2 的两种似然模式
    W1, W2 = model.get_W1_W2()
    theta1 = model.theta1
    Hs = model.sample_H()  # (S, n, K)
    loglik1 = []
    loglik2 = []
    if model.likelihood_x2 == 'nb':
        lambda2_nb = model.lambda2_nb
    else:
        lambda2_ber = model.lambda2_ber
        X2_bin = (X2 > 0).to(X2.dtype)
    for t in range(Hs.size(0)):
        H = Hs[t]
        Y1 = torch.exp(H) @ W1
        Y2 = torch.exp(H) @ W2
        loglik1.append(nb_log_prob(X1, Y1, theta1).sum())
        if model.likelihood_x2 == 'nb':
            loglik2.append(nb_log_prob(X2, Y2, lambda2_nb).sum())
        else:
            p2 = torch.clamp(Y2 * lambda2_ber, min=1e-8, max=1.0 - 1e-8)
            loglik2.append(bernoulli_log_prob(X2_bin, p2).sum())
    loglik1 = torch.stack(loglik1).mean()
    loglik2 = torch.stack(loglik2).mean()
    return loglik1, loglik2

num_steps = 2000
print_every = 100
for step in range(1, num_steps + 1):
    opt.zero_grad()
    loss = model(X1_t, X2_t)  # 负ELBO
    loss.backward()
    opt.step()

    if step == 1 or step % print_every == 0:
        with torch.no_grad():
            elbo_val = model.elbo(X1_t, X2_t)
            kl_val = model.compute_KL_u()
            loglik1, loglik2 = compute_loglik_terms(model, X1_t, X2_t)
            print(f"[step {step:03d}] loss={loss.item():.3f}  ELBO={elbo_val.item():.3f}  KL={kl_val.item():.3f}  "
                  f"loglik1={loglik1.item():.3f}  loglik2={loglik2.item():.3f}")

print('Training finished.')

_LinAlgError: linalg.cholesky: The factorization could not be completed because the input is not positive-definite (the leading minor of order 1 is not positive-definite).

In [ ]:
# （保留空白单元，已移除诊断相关内容）
